# 파일 만들기

In [51]:
import os
import csv

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from xgboost import XGBClassifier 

import joblib

In [43]:
data = pd.read_csv('./hand_data.csv', header=None)
print(data.shape)
data.head()

(35, 64)


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.611172,0.770672,4.027141e-07,0.582887,0.739500,-0.017788,0.563058,0.686377,-0.030466,0.581954,...,0.650544,0.646972,-0.064163,0.630179,0.679668,-0.061600,0.620859,0.706307,-0.054114,scissors
1,0.597230,0.651086,-3.601817e-07,0.559453,0.627323,-0.005406,0.530490,0.579848,-0.006688,0.531568,...,0.612108,0.515366,-0.020866,0.608801,0.547380,-0.012378,0.618022,0.554529,-0.003282,rock
2,0.607775,0.631376,2.167319e-07,0.569666,0.599787,-0.010450,0.540896,0.548929,-0.014367,0.521067,...,0.686142,0.474362,-0.037437,0.703677,0.449353,-0.044198,0.717858,0.424703,-0.049064,paper
3,0.625219,0.645236,-3.786552e-07,0.585828,0.615293,0.006780,0.567804,0.578014,0.007071,0.555954,...,0.574762,0.515083,-0.025344,0.577575,0.546231,-0.022966,0.595921,0.551566,-0.018652,rock
4,0.610893,0.639902,-3.480116e-07,0.571525,0.633334,-0.005433,0.534293,0.603938,-0.007969,0.524923,...,0.594749,0.508148,-0.024952,0.598476,0.539715,-0.017081,0.610598,0.542832,-0.007855,rock


In [44]:
columns = []

# 21 * 3 = 63 + 1(label)
for i in range(21):
    columns.extend([f"x{i}",f"y{i}",f"z{i}"]) 

columns.append('label')
data.columns = columns
data.head()

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,x18,y18,z18,x19,y19,z19,x20,y20,z20,label
0,0.611172,0.770672,4.027141e-07,0.582887,0.739500,-0.017788,0.563058,0.686377,-0.030466,0.581954,...,0.650544,0.646972,-0.064163,0.630179,0.679668,-0.061600,0.620859,0.706307,-0.054114,scissors
1,0.597230,0.651086,-3.601817e-07,0.559453,0.627323,-0.005406,0.530490,0.579848,-0.006688,0.531568,...,0.612108,0.515366,-0.020866,0.608801,0.547380,-0.012378,0.618022,0.554529,-0.003282,rock
2,0.607775,0.631376,2.167319e-07,0.569666,0.599787,-0.010450,0.540896,0.548929,-0.014367,0.521067,...,0.686142,0.474362,-0.037437,0.703677,0.449353,-0.044198,0.717858,0.424703,-0.049064,paper
3,0.625219,0.645236,-3.786552e-07,0.585828,0.615293,0.006780,0.567804,0.578014,0.007071,0.555954,...,0.574762,0.515083,-0.025344,0.577575,0.546231,-0.022966,0.595921,0.551566,-0.018652,rock
4,0.610893,0.639902,-3.480116e-07,0.571525,0.633334,-0.005433,0.534293,0.603938,-0.007969,0.524923,...,0.594749,0.508148,-0.024952,0.598476,0.539715,-0.017081,0.610598,0.542832,-0.007855,rock


In [45]:
data['label'].value_counts()

label
scissors    15
paper       11
rock         9
Name: count, dtype: int64

In [46]:
def encoding(label):
    if label == 'rock':
        return 0
    elif label == 'scissors':
        return 1
    elif label == 'paper':
        return 2

data['label_enc'] = data['label'].apply(lambda x : encoding(x))
data.head()

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,y18,z18,x19,y19,z19,x20,y20,z20,label,label_enc
0,0.611172,0.770672,4.027141e-07,0.582887,0.739500,-0.017788,0.563058,0.686377,-0.030466,0.581954,...,0.646972,-0.064163,0.630179,0.679668,-0.061600,0.620859,0.706307,-0.054114,scissors,1
1,0.597230,0.651086,-3.601817e-07,0.559453,0.627323,-0.005406,0.530490,0.579848,-0.006688,0.531568,...,0.515366,-0.020866,0.608801,0.547380,-0.012378,0.618022,0.554529,-0.003282,rock,0
2,0.607775,0.631376,2.167319e-07,0.569666,0.599787,-0.010450,0.540896,0.548929,-0.014367,0.521067,...,0.474362,-0.037437,0.703677,0.449353,-0.044198,0.717858,0.424703,-0.049064,paper,2
3,0.625219,0.645236,-3.786552e-07,0.585828,0.615293,0.006780,0.567804,0.578014,0.007071,0.555954,...,0.515083,-0.025344,0.577575,0.546231,-0.022966,0.595921,0.551566,-0.018652,rock,0
4,0.610893,0.639902,-3.480116e-07,0.571525,0.633334,-0.005433,0.534293,0.603938,-0.007969,0.524923,...,0.508148,-0.024952,0.598476,0.539715,-0.017081,0.610598,0.542832,-0.007855,rock,0


In [57]:
X = data.iloc[:, :-2]
y = data['label_enc']
print(X.shape, y.shape)
print(y)

(35, 63) (35,)
0     1
1     0
2     2
3     0
4     0
5     0
6     0
7     0
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    2
18    2
19    2
20    2
21    2
22    2
23    2
24    2
25    0
26    0
27    0
28    1
29    2
30    2
31    1
32    1
33    1
34    1
Name: label_enc, dtype: int64


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (28, 63), y_train: (28,), X_test: (7, 63), y_test: (7,)


In [59]:
model = XGBClassifier(n_estimators=100, random_state=42)
model.fit(np.array(X_train), np.array(y_train))

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [52]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      0.50      0.67         2
           2       0.80      1.00      0.89         4

    accuracy                           0.86         7
   macro avg       0.93      0.83      0.85         7
weighted avg       0.89      0.86      0.84         7



In [54]:
joblib.dump(model, './rcp.pkl')

['./rcp.pkl']

In [55]:
joblib.load('./rcp.pkl')

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None
